In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt

from tensorflow import keras
from sklearn.model_selection import train_test_split

print(tf.__version__)
print(keras.__version__)

In [2]:
tf.test.is_gpu_available()

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

In [4]:
import tensorflow.keras.applications.resnet50 as resnet50 
import tensorflow.keras.applications.vgg19 as vgg19
import tensorflow.keras.applications.densenet as densenet
import tensorflow.keras.applications.inception_v3 as inception_v3

In [5]:
import tensorflow.keras.preprocessing.image as image
import tensorflow.keras.losses as losses
import tensorflow.keras.layers as layers
import tensorflow.keras.models as models
import tensorflow.keras.callbacks as callbacks
import tensorflow.keras.backend as K

# Dataset

In [6]:
from scipy.io import loadmat
from os.path import join

prefix = './Stanford_Dog_Breed'
imgfix = 'Images'
anofix = 'Annotation'

In [7]:
ImgTrainGen = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input, 
                                  width_shift_range=0.2, 
                                  height_shift_range=0.2, 
                                  shear_range=0.2, 
                                  zoom_range=0.2, 
                                  horizontal_flip=True, 
#                                   validation_split=1/6.
                                      )

Train = ImgTrainGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')

# Valid = ImgTrainGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
#                                    target_size=(224, 224), 
#                                    class_mode='sparse', 
#                                    batch_size=32, 
#                                    shuffle=True, 
#                                    seed=None, 
#                                    subset='validation', 
#                                    interpolation='nearest')


ImgTestGen = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input, validation_split=1/6.)

Test  = ImgTestGen.flow_from_directory(join(prefix, imgfix, 'Test' ), 
                                       target_size=(224, 224), 
                                       class_mode='sparse', 
                                       batch_size=32, 
                                       shuffle=False, 
                                       interpolation='nearest')



Found 12000 images belonging to 120 classes.
Found 8580 images belonging to 120 classes.


In [8]:
def MultiTaskDataGen(ImgGen):
    while True:
        X = ImgGen.next()
        yield [X[0], X[1]], [X[1], X[1], X[1]]
        
TrainData = MultiTaskDataGen(Train)
# ValidData = MultiTaskDataGen(Valid)
TestData  = MultiTaskDataGen(Test)

In [ ]:
Test.class_indices

def gen_path(idx):
    try:    path = join(prefix, imgfix, ftest[idx][0][0])
    except: path = join(prefix, imgfix, ftest[-1][0][0])      
    return  path

def gen_labl(idx):
    try:    labl = ltest[idx][0]
    except: labl = ltest[-1][0]
    return  labl

def get_imgs(idx):
    img = image.load_img(gen_path(idx), target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    
    return x

## Data Preparation

XTrain = []
for idx in range(ntrain):
    path = join(prefix, imgfix, ftrain[idx][0][0])
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        XTrain.append(resnet50.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')

XTrain = np.vstack(XTrain)

XTest = []
for idx in range(ntest):
    path = gen_path(idx)
    try:
        img = image.load_img(path, target_size=(224, 224))
        x = image.img_to_array(img)
        x = np.expand_dims(x, axis=0)
        XTest.append(resnet50.preprocess_input(x))
    except:
        print('Error at processing the', idx, 'images.')
        
XTest = np.vstack(XTest)

# Models

## Classify using ImageNet Classes with ResNet50

In [ ]:
resnet50_naive = resnet50.ResNet50(weights='imagenet')
# YTest = resnet50_naive.predict(XTest)

In [ ]:
resnet50_naive.compile('SGD', loss=losses.categorical_crossentropy, metrics={'output_a': 'accuracy'})

In [ ]:
YTest_ = resnet50_naive.evaluate_generator(Test)

In [ ]:
YTest_

In [ ]:
Test.total_batches_seen

In [ ]:
print XTest[0:1000, :, :, :].shape
print YTest.shape

In [ ]:
# YTest = resnet50.decode_predictions(YTest, top=3)[0]

correct, top = 0, 1

for idx in range(ntest):
    pred = resnet50.decode_predictions(YTest[idx:idx+1,:], top=top)[0]
    for k in range(top):
        if pred[k][1] not in DogBreed:
            continue
        labl = DogBreed[pred[k][1]]
        if labl == gen_labl(idx):
            correct += 1
            break


print(correct)
print(correct / float(ntest))
    

## InceptionV3

In [9]:
inceptionv3_naive = inception_v3.InceptionV3(weights='imagenet', include_top=False)

## Naive Model Plug with FC120 

In [57]:
# Inputs 
input_image = inceptionv3_naive.input

# Intermediate Layers
x = inceptionv3_naive.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dropout(0.2)(x)
# x = layers.Dense(240, activation='relu')(x)
x = layers.Dense(120, activation='softmax')(x)

# Compile
model = models.Model(inputs=input_image, outputs=x)
for layer in inceptionv3_naive.layers:
    layer.trainable = False

model.compile(
    optimizer='SGD', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

# Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'inceptionv3_dfc120_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res
            

In [58]:
model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=201, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval()]
)

Epoch 1/201
80/80 [==============================] - 13s 157ms/step - loss: 4.8375 - acc: 0.0172
Epoch 2/201
80/80 [==============================] - 9s 109ms/step - loss: 4.5872 - acc: 0.0469
Epoch 3/201
80/80 [==============================] - 9s 114ms/step - loss: 4.3625 - acc: 0.1047
Epoch 4/201
79/80 [============================>.] - ETA: 0s - loss: 4.1474 - acc: 0.1812
 [2.9085619246209418, 0.43228438228438226]
80/80 [==============================] - 42s 527ms/step - loss: 4.1454 - acc: 0.1820
Epoch 5/201
80/80 [==============================] - 10s 122ms/step - loss: 3.9823 - acc: 0.2223
Epoch 6/201
80/80 [==============================] - 10s 121ms/step - loss: 3.7589 - acc: 0.2887
Epoch 7/201
79/80 [============================>.] - ETA: 0s - loss: 3.5903 - acc: 0.3192
 [1.8556325831891218, 0.6206293706293706]
80/80 [==============================] - 40s 494ms/step - loss: 3.5878 - acc: 0.3188
Epoch 8/201
80/80 [==============================] - 9s 111ms/step - loss: 3.4401 

79/80 [============================>.] - ETA: 0s - loss: 1.6714 - acc: 0.5993
 [0.7356553755285807, 0.7987179487179488]
80/80 [==============================] - 40s 498ms/step - loss: 1.6680 - acc: 0.5996
Epoch 62/201
80/80 [==============================] - 9s 113ms/step - loss: 1.6546 - acc: 0.5980
Epoch 63/201
80/80 [==============================] - 9s 113ms/step - loss: 1.6866 - acc: 0.5906
Epoch 64/201
79/80 [============================>.] - ETA: 0s - loss: 1.5901 - acc: 0.6203
 [0.7465982511389465, 0.7947552447552447]
80/80 [==============================] - 38s 480ms/step - loss: 1.5904 - acc: 0.6203
Epoch 65/201
80/80 [==============================] - 9s 113ms/step - loss: 1.6665 - acc: 0.5953
Epoch 66/201
80/80 [==============================] - 10s 122ms/step - loss: 1.6424 - acc: 0.6105
Epoch 67/201
79/80 [============================>.] - ETA: 0s - loss: 1.6143 - acc: 0.6056
 [0.7416082740501712, 0.7987179487179488]
80/80 [==============================] - 39s 484ms/step

80/80 [==============================] - 9s 112ms/step - loss: 1.4772 - acc: 0.6270
Epoch 121/201
79/80 [============================>.] - ETA: 0s - loss: 1.4206 - acc: 0.6297
 [0.7988863774123183, 0.7982517482517483]
80/80 [==============================] - 38s 480ms/step - loss: 1.4322 - acc: 0.6281
Epoch 122/201
80/80 [==============================] - 10s 123ms/step - loss: 1.4655 - acc: 0.6203
Epoch 123/201
80/80 [==============================] - 9s 115ms/step - loss: 1.4175 - acc: 0.6402
Epoch 124/201
79/80 [============================>.] - ETA: 0s - loss: 1.4502 - acc: 0.6282
 [0.8022352628364646, 0.7977855477855478]
80/80 [==============================] - 39s 484ms/step - loss: 1.4504 - acc: 0.6281
Epoch 125/201
80/80 [==============================] - 9s 112ms/step - loss: 1.4234 - acc: 0.6480
Epoch 126/201
80/80 [==============================] - 9s 112ms/step - loss: 1.4849 - acc: 0.6121
Epoch 127/201
79/80 [============================>.] - ETA: 0s - loss: 1.3845 - acc: 

80/80 [==============================] - 10s 123ms/step - loss: 1.3276 - acc: 0.6457
Epoch 180/201
80/80 [==============================] - 9s 112ms/step - loss: 1.3169 - acc: 0.6477
Epoch 181/201
79/80 [============================>.] - ETA: 0s - loss: 1.3476 - acc: 0.6456
 [0.8278498791053893, 0.8004662004662004]
80/80 [==============================] - 38s 480ms/step - loss: 1.3500 - acc: 0.6453
Epoch 182/201
80/80 [==============================] - 9s 112ms/step - loss: 1.3688 - acc: 0.6414
Epoch 183/201
80/80 [==============================] - 10s 123ms/step - loss: 1.3219 - acc: 0.6477
Epoch 184/201
79/80 [============================>.] - ETA: 0s - loss: 1.3382 - acc: 0.6472
 [0.838619410536265, 0.798018648018648]
80/80 [==============================] - 38s 481ms/step - loss: 1.3433 - acc: 0.6465
Epoch 185/201
80/80 [==============================] - 9s 112ms/step - loss: 1.3307 - acc: 0.6449
Epoch 186/201
80/80 [==============================] - 9s 112ms/step - loss: 1.3091 - 

In [83]:
ImgGen = image.ImageDataGenerator(preprocessing_function=inception_v3.preprocess_input, 
                                  validation_split=1/6.)

TrainF = ImgGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='training', 
                                   interpolation='nearest')
ValidF = ImgGen.flow_from_directory(join(prefix, imgfix, 'Train'), 
                                   target_size=(224, 224), 
                                   class_mode='sparse', 
                                   batch_size=32, 
                                   shuffle=True, 
                                   seed=None, 
                                   subset='validation', 
                                   interpolation='nearest')

Found 10080 images belonging to 120 classes.
Found 1920 images belonging to 120 classes.


In [84]:
model = models.load_model('inceptionv3_dfc120_114_8031')

In [85]:
for layer in model.layers:
    layer.trainable = False
for layer in model.layers[279:]:
    layer.trainable = True

model.compile(
    optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9), 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

# Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'inceptionv3_dfc120_ft_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res

In [81]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, None, None, 3 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, None, 3 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, None, 3 0           batch_normalization[0][0]        
__________________________________________________________________________________________________
conv2d_1 (

In [86]:
model.fit_generator(
    TrainF, 
    steps_per_epoch=80, 
    epochs=41, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval()]
)

Epoch 1/41
80/80 [==============================] - 17s 212ms/step - loss: 1.3406 - acc: 0.6629
Epoch 2/41
80/80 [==============================] - 10s 122ms/step - loss: 1.3778 - acc: 0.6492
Epoch 3/41
80/80 [==============================] - 10s 122ms/step - loss: 1.3596 - acc: 0.6508
Epoch 4/41
79/80 [============================>.] - ETA: 0s - loss: 1.3367 - acc: 0.6543
 [0.7473589497563257, 0.8005827505827506]
80/80 [==============================] - 98s 1s/step - loss: 1.3304 - acc: 0.6562
Epoch 5/41
80/80 [==============================] - 10s 123ms/step - loss: 1.2873 - acc: 0.6680
Epoch 6/41
80/80 [==============================] - 10s 123ms/step - loss: 1.2797 - acc: 0.6746
Epoch 7/41
79/80 [============================>.] - ETA: 0s - loss: 1.2775 - acc: 0.6744
 [0.7494967815984633, 0.7983682983682984]
80/80 [==============================] - 39s 485ms/step - loss: 1.2761 - acc: 0.6750
Epoch 8/41
80/80 [==============================] - 11s 133ms/step - loss: 1.2481 - acc: 0.

inceptionv3_modified = models.load_model('fc_d240120_20')
inceptionv3_modified.compile(
        optimizer='SGD', 
        loss='categorical_crossentropy', 
        metrics=['accuracy'])

In [ ]:
inceptionv3_modified.summary()

In [ ]:
print(inceptionv3_modified.evaluate_generator(Train, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )
print(inceptionv3_modified.evaluate_generator(Valid, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )
print(inceptionv3_modified.evaluate_generator(Test,  verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )

In [ ]:
inceptionv3_modified.fit_generator(
    Train, 
    steps_per_epoch=312, 
    epochs=41, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
#     validation_data=Valid, 
#     validation_steps=60, 
    initial_epoch=21,
    callbacks=[SaveModelandEval()])

In [ ]:
input_image = inceptionv3_naive.input

x = layers.GlobalAveragePooling2D()(inceptionv3_naive.output)
print(x.shape)

f_1 = layers.Dense(64)(x)
f_2 = layers.Dense(64, activation='sigmoid')(x)
feature = layers.multiply([f_1, f_2])
print(feature.shape)

predict = layers.Dense(120, activation='sigmoid', name='softmax1')(feature)
auxiliary = layers.Dense(120, activation='sigmoid', name='auxiliary')(x)

print(predict.shape)
print(auxiliary.shape)

In [ ]:
input_target = layers.Input(shape=(1,))
centers = layers.Embedding(120, 64)(input_target)
l2_loss = layers.Lambda(lambda x: K.sum(K.square(x[0]-x[1][:,0]), 1, keepdims=True), name='l2')([feature, centers])

print(input_target.shape)
print(centers.shape)
print(l2_loss.shape)

## Feature Concatation

In [97]:
# Inputs 
input_image = inceptionv3_naive.input

# Intermediate Layers
x1 = inceptionv3_naive.output
x1 = layers.GlobalAveragePooling2D()(x1)
# x2 = inceptionv3_naive.get_layer('mixed7').output
x2 = inceptionv3_naive.get_layer('mixed9').output
x2 = layers.GlobalAveragePooling2D()(x2)
x = layers.Concatenate(axis=-1)([x1,x2])
x = layers.Dropout(0.4)(x)
x = layers.Dense(120, activation='softmax')(x)

# Compile
model = models.Model(inputs=input_image, outputs=x)
for layer in inceptionv3_naive.layers:
    layer.trainable = False

model.compile(
    optimizer='SGD', 
    loss='sparse_categorical_crossentropy', 
    metrics=['accuracy'])

# Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = list()
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'inceptionv3_d2fc120_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res
            

In [98]:
model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=201, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=0,
    callbacks=[SaveModelandEval()]
)

Epoch 1/201
80/80 [==============================] - 16s 203ms/step - loss: 4.7924 - acc: 0.0172
Epoch 2/201
80/80 [==============================] - 9s 110ms/step - loss: 4.5486 - acc: 0.0496
Epoch 3/201
80/80 [==============================] - 9s 116ms/step - loss: 4.3404 - acc: 0.1258
Epoch 4/201
79/80 [============================>.] - ETA: 0s - loss: 4.1264 - acc: 0.1748
 [2.690066069267291, 0.5432400932400933]
80/80 [==============================] - 46s 580ms/step - loss: 4.1247 - acc: 0.1758
Epoch 5/201
80/80 [==============================] - 10s 126ms/step - loss: 3.8846 - acc: 0.2668
Epoch 6/201
80/80 [==============================] - 9s 115ms/step - loss: 3.6992 - acc: 0.3031
Epoch 7/201
79/80 [============================>.] - ETA: 0s - loss: 3.4904 - acc: 0.3592
 [1.6523953325126, 0.6815850815850816]
80/80 [==============================] - 40s 497ms/step - loss: 3.4870 - acc: 0.3602
Epoch 8/201
80/80 [==============================] - 9s 112ms/step - loss: 3.3593 - acc:

79/80 [============================>.] - ETA: 0s - loss: 1.6724 - acc: 0.5870
 [0.7392309925202112, 0.7917249417249417]
80/80 [==============================] - 40s 494ms/step - loss: 1.6709 - acc: 0.5863
Epoch 62/201
80/80 [==============================] - 9s 113ms/step - loss: 1.6291 - acc: 0.5906
Epoch 63/201
80/80 [==============================] - 9s 113ms/step - loss: 1.6331 - acc: 0.6043
Epoch 64/201
79/80 [============================>.] - ETA: 0s - loss: 1.6215 - acc: 0.6036
 [0.7334173941463482, 0.7970862470862471]
80/80 [==============================] - 39s 493ms/step - loss: 1.6239 - acc: 0.6035
Epoch 65/201
80/80 [==============================] - 9s 112ms/step - loss: 1.6142 - acc: 0.6039
Epoch 66/201
80/80 [==============================] - 10s 124ms/step - loss: 1.6407 - acc: 0.5949
Epoch 67/201
79/80 [============================>.] - ETA: 0s - loss: 1.5952 - acc: 0.6088
 [0.74178794795008, 0.7956876456876457]
80/80 [==============================] - 39s 489ms/step -

80/80 [==============================] - 9s 113ms/step - loss: 1.4521 - acc: 0.6332
Epoch 121/201
79/80 [============================>.] - ETA: 0s - loss: 1.4609 - acc: 0.6206
 [0.7865523778525179, 0.7983682983682984]
80/80 [==============================] - 39s 486ms/step - loss: 1.4588 - acc: 0.6215
Epoch 122/201
80/80 [==============================] - 10s 125ms/step - loss: 1.4255 - acc: 0.6332
Epoch 123/201
80/80 [==============================] - 9s 114ms/step - loss: 1.4301 - acc: 0.6426
Epoch 124/201
79/80 [============================>.] - ETA: 0s - loss: 1.4390 - acc: 0.6341
 [0.7911933501392253, 0.7982517482517483]
80/80 [==============================] - 40s 494ms/step - loss: 1.4400 - acc: 0.6340
Epoch 125/201
80/80 [==============================] - 9s 113ms/step - loss: 1.3935 - acc: 0.6324
Epoch 126/201
80/80 [==============================] - 9s 113ms/step - loss: 1.3852 - acc: 0.6387
Epoch 127/201
79/80 [============================>.] - ETA: 0s - loss: 1.4531 - acc: 

Epoch 179/201
80/80 [==============================] - 10s 130ms/step - loss: 1.3292 - acc: 0.6473
Epoch 180/201
80/80 [==============================] - 10s 119ms/step - loss: 1.3509 - acc: 0.6539
Epoch 181/201
79/80 [============================>.] - ETA: 0s - loss: 1.3071 - acc: 0.6511
 [0.8269413478832236, 0.7997668997668997]
80/80 [==============================] - 39s 490ms/step - loss: 1.3030 - acc: 0.6523
Epoch 182/201
80/80 [==============================] - 9s 113ms/step - loss: 1.3479 - acc: 0.6469
Epoch 183/201
80/80 [==============================] - 10s 124ms/step - loss: 1.3126 - acc: 0.6512
Epoch 184/201
79/80 [============================>.] - ETA: 0s - loss: 1.2444 - acc: 0.6752
 [0.8241954635696714, 0.8004662004662004]
80/80 [==============================] - 39s 489ms/step - loss: 1.2489 - acc: 0.6746
Epoch 185/201
80/80 [==============================] - 9s 113ms/step - loss: 1.3242 - acc: 0.6578
Epoch 186/201
80/80 [==============================] - 9s 113ms/step 

## Model with the GLU activations

In [54]:
## Input
input_image = inceptionv3_naive.input

## Intermediate Layers
x = layers.GlobalAveragePooling2D()(inceptionv3_naive.output)
x = layers.Dropout(0.3)(x)

f_1 = layers.Dense(240)(x)
f_2 = layers.Dense(240, activation='sigmoid')(x)
feature = layers.multiply([f_1, f_2])

predict = layers.Dense(120, activation='sigmoid', name='softmax1')(feature)

print(x.shape)
print(feature.shape)
print(predict.shape)

## Compile 
model = models.Model(inputs=[input_image], outputs=[predict])
for layer in inceptionv3_naive.layers:
    layer.trainable = False
model.compile(
    optimizer='adam', 
    loss=['sparse_categorical_crossentropy'], 
    metrics=['accuracy']
)

## Callbacks
class SaveModelandEval(callbacks.Callback):
    prev_res = []
    def __init__(self):
        self.prev_res = [0., 0.]
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 3 == 0:
            res = model.evaluate_generator(Test, verbose=0, steps=None, workers=4, use_multiprocessing=True, max_queue_size=12)
            print('\n', res)
            if res[1] > self.prev_res[1]:
                models.save_model(model, 'inceptionv3_dglu240fc120_%s_%s'%(epoch, int(res[1]*10000)) )
                self.prev_res = res
                

(?, 2048)
(?, 240)
(?, 120)


In [56]:
model.fit_generator(
    Train, 
    steps_per_epoch=80, 
    epochs=201, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=101,
    callbacks=[SaveModelandEval()]
)

Epoch 102/201
80/80 [==============================] - 10s 127ms/step - loss: 1.2371 - acc: 0.6559
Epoch 103/201
79/80 [============================>.] - ETA: 0s - loss: 1.2399 - acc: 0.6428
 [1.106701986140801, 0.7663170163170163]
80/80 [==============================] - 39s 486ms/step - loss: 1.2447 - acc: 0.6414
Epoch 104/201
80/80 [==============================] - 9s 112ms/step - loss: 1.2617 - acc: 0.6555
Epoch 105/201
80/80 [==============================] - 9s 112ms/step - loss: 1.3364 - acc: 0.6172
Epoch 106/201
79/80 [============================>.] - ETA: 0s - loss: 1.2025 - acc: 0.6574
 [1.221794537927791, 0.7648018648018649]
80/80 [==============================] - 39s 490ms/step - loss: 1.2050 - acc: 0.6562
Epoch 107/201
80/80 [==============================] - 9s 112ms/step - loss: 1.2231 - acc: 0.6586
Epoch 108/201
80/80 [==============================] - 9s 112ms/step - loss: 1.2223 - acc: 0.6457
Epoch 109/201
79/80 [============================>.] - ETA: 0s - loss: 1.

Epoch 161/201
80/80 [==============================] - 9s 113ms/step - loss: 1.2168 - acc: 0.6527
Epoch 162/201
80/80 [==============================] - 10s 123ms/step - loss: 1.2138 - acc: 0.6484
Epoch 163/201
79/80 [============================>.] - ETA: 0s - loss: 1.1577 - acc: 0.6792
 [1.2131502959635603, 0.7615384615384615]
80/80 [==============================] - 39s 487ms/step - loss: 1.1588 - acc: 0.6793
Epoch 164/201
80/80 [==============================] - 9s 113ms/step - loss: 1.1966 - acc: 0.6516
Epoch 165/201
80/80 [==============================] - 9s 112ms/step - loss: 1.2092 - acc: 0.6652
Epoch 166/201
79/80 [============================>.] - ETA: 0s - loss: 1.2365 - acc: 0.6527
 [1.2548714658587945, 0.7637529137529138]
80/80 [==============================] - 39s 488ms/step - loss: 1.2334 - acc: 0.6535
Epoch 167/201
80/80 [==============================] - 10s 125ms/step - loss: 1.1807 - acc: 0.6641
Epoch 168/201
80/80 [==============================] - 10s 121ms/step 

In [ ]:
print(model.evaluate_generator(Train, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )
print(model.evaluate_generator(Valid, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )



## GLU + Center Loss

In [ ]:
## Input
input_image = inceptionv3_naive.input
input_target = layers.Input(shape=(1,))

## Intermediate Layers
x = layers.GlobalAveragePooling2D()(inceptionv3_naive.output)

f_1 = layers.Dense(64)(x)
f_2 = layers.Dense(64, activation='sigmoid')(x)
feature = layers.multiply([f_1, f_2])

predict = layers.Dense(120, activation='sigmoid', name='softmax')(feature)
auxiliary = layers.Dense(120, activation='sigmoid', name='auxiliary')(x)

print(x.shape)
print(feature.shape)
print(predict.shape)
print(auxiliary.shape)

centers = layers.Embedding(120, 64)(input_target)
l2_loss = layers.Lambda(lambda x: K.sum(K.square(x[0]-x[1][:,0]), 1, keepdims=True), name='l2')([feature, centers])

print(centers.shape)
print(l2_loss.shape)

## Compile 
model = models.Model(inputs=[input_image, input_target], outputs=[predict, l2_loss, auxiliary])
for layer in inceptionv3_naive.layers:
    layer.trainable = False
model.compile(
    optimizer='adam', 
    loss=['sparse_categorical_crossentropy', lambda y_true, y_pred: y_pred, 'sparse_categorical_crossentropy'], 
    loss_weights=[1., 0.25, 0.25], 
    metrics={'softmax':'accuracy','auxiliary':'accuracy'}
)

## Callbacks
class SaveModelandEval(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 5 == 0:
            models.save_model(model, 'inceptionv3_glu64c_120_'+str(epoch) )
            print(model.evaluate_generator(TestData, steps=269, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12) )
        

In [ ]:
model = models.load_model('inceptionv3_glu64c_120_20',
                          custom_objects={'<lambda>': lambda y_true, y_pred: y_pred})

for layer in inceptionv3_naive.layers:
    layer.trainable = False
model.compile(
    optimizer=keras.optimizers.SGD(lr=1e-4, momentum=0.9), 
    loss=['sparse_categorical_crossentropy', lambda y_true, y_pred: y_pred, 'sparse_categorical_crossentropy'], 
    loss_weights=[1., 0.25, 0.25], 
    metrics={'softmax':'accuracy','auxiliary':'accuracy'}
)
model.evaluate_generator(TestData, steps=269, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12)

In [ ]:
model.fit_generator(
    TrainData, 
    steps_per_epoch=312, 
    epochs=41, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=20,
    initial_epoch=21,
    callbacks=[SaveModelandEval()]
)

### Save the trained models

In [ ]:
models.save_model(inceptionv3_modified, 'sample_model_name')

## Fine Tunning Demo

In [ ]:
inceptionv3_modified = models.load_model('fc12040')

In [89]:
# Get the layers infomation
for i in range(len(inceptionv3_naive.layers)):
    layer = inceptionv3_naive.layers[i]
    layerType = str(layer.name)
    if layerType[:5] == 'mixed':
        print(layerType, i)

print('\n\n\nBegin Configuring...\n\n\n')
        
# # Set the paramters of the last InceptionBlock to be trainable
# for layer in inceptionv3_modified.layers[279:]:
#     print(str(layer.name) )
#     layer.trainable = True
    
# inceptionv3_modified.compile(
#     optimizer=keras.optimizers.SGD(lr=0.0001, momentum=0.9), 
#     loss='categorical_crossentropy', 
#     metrics=['accuracy'])
                             
# inceptionv3_modified.summary()

mixed0 40
mixed1 63
mixed2 86
mixed3 100
mixed4 132
mixed5 164
mixed6 196
mixed7 228
mixed8 248
mixed9_0 276
mixed9 279
mixed9_1 307
mixed10 310



Begin Configuring...





1. Each mixed layer split the InceptionBlock within the models. 
2. Start by fine tuning the last IncptionBlock. 


In [ ]:
class SaveModelandEval(callbacks.Callback):
    def on_epoch_end(self, epoch, logs):
        if epoch == 0: return
        if epoch % 20 == 0:
            models.save_model(inceptionv3_modified, 'fc120_ft_'+str(epoch) )
            print(inceptionv3_modified.evaluate_generator(Test,  verbose=1) )

In [ ]:
inceptionv3_modified = models.load_model('fc120_ft_20')

In [ ]:
inceptionv3_modified.summary()

In [ ]:
inceptionv3_modified.fit_generator(
    Train, 
    steps_per_epoch=312, 
    epochs=101, 
    verbose=1, 
    workers=4,
    use_multiprocessing=True,
    max_queue_size=12,
    validation_data=Valid, 
    validation_steps=60, 
    initial_epoch=61,
    callbacks=[SaveModelandEval()])

In [ ]:
sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))

In [ ]:
print(inceptionv3_modified.evaluate_generator(Train, verbose=1) )
print(inceptionv3_modified.evaluate_generator(Valid, verbose=1) )
print(inceptionv3_modified.evaluate_generator(Test,  verbose=1) )

In [ ]:
def decode_prediction(Pred, Top=1):
    res = []
    for pred in Pred:
        scores = [(pred[i], i) for i in range(120)]
        scores.sort(reverse=1)
        res.append(scores[:Top])
    return res

def calculate_accuracy(Pred, Lbl, Top):
    idx, cnt = 0, 0
    for res in decode_prediction(Pred, Top):
        lbls = [res[i][1] for i in range(Top)]
        if Lbl[idx] in lbls:
            cnt += 1
        idx += 1
    return cnt / len(Pred)

In [ ]:
inceptionv3_modified = models.load_model('fc120_ft_40')

In [ ]:
P = inceptionv3_modified.predict_generator(Test, verbose=1, workers=4, use_multiprocessing=True, max_queue_size=12)


In [ ]:
# 20 epochs
print(calculate_accuracy(P, Test.classes, Top=1) )
print(calculate_accuracy(P, Test.classes, Top=2) )

In [ ]:
# 40 epochs
print(calculate_accuracy(P, Test.classes, Top=1) )
print(calculate_accuracy(P, Test.classes, Top=2) )

In [ ]:
# fine tune for 20 epochs
print(calculate_accuracy(P, Test.classes, Top=1) )
print(calculate_accuracy(P, Test.classes, Top=2) )

In [ ]:
# fine tune for 40 epochs
print(calculate_accuracy(P, Test.classes, Top=1) )
print(calculate_accuracy(P, Test.classes, Top=2) )

In [ ]:
Xception = keras.applications.Xception(weights='imagenet', include_top=False)

In [ ]:
Xception.output.shape